In [1]:
# download the dataset and save it in the same dir with this file.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
kf = KFold(n_splits=10)
# import scikit learn
from sklearn import linear_model,svm
from sklearn.preprocessing import PolynomialFeatures
folder = ['datasets/' + chr(ord('A') + x) for x in range(18)]

In [2]:
def getxandy(filename):
  with open(filename,'r') as f:
    df = pd.read_csv(f,index_col=0)
    x = np.zeros((len(df.x),2))
    y = np.zeros((len(df.x)))
    x[:,0] = df.x.to_numpy()
    x[:,1] = df.y.to_numpy()
    y = df.z.to_numpy()
  return x,y

In [3]:
def scoreperdataset(y_pred,y_test):
  return np.sqrt(np.sum(np.square(y_pred - y_test)/len(y_pred)))

In [4]:
finalacc = []
for f in folder:
  X,y = getxandy(f+'/train.csv')
  reg = linear_model.Ridge(alpha=1.)
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss = scoreperdataset(y_pred, y_test) 
  finalacc.append(loss)
  print('loss for f is ', loss)
print('final average is ', sum(finalacc)/len(finalacc))

loss for f is  1.1836335580257547
loss for f is  1.181355786543824
loss for f is  1.1878460325292337
loss for f is  1.024955748318806
loss for f is  1.0187753224764637
loss for f is  1.0356085656179976
loss for f is  0.8707129839343839
loss for f is  0.8792557862570307
loss for f is  2.6399452500067713
loss for f is  1.3167897470892946
loss for f is  1.2872594525532195
loss for f is  1.2829418193064346
loss for f is  1.1617367384144413
loss for f is  1.1769813889101486
loss for f is  1.1913125994529912
loss for f is  1.2262781505761402
loss for f is  1.1201550328613323
loss for f is  5.817090552510703
final average is  1.4779241397436096


In [5]:
model = Pipeline([('poly', PolynomialFeatures(degree=3)),('linear', GradientBoostingRegressor())])

In [ ]:
finalacc = []
for f in folder:
  X,y = getxandy(f+'/train.csv')
  reg = model
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss = scoreperdataset(y_pred, y_test) 
  finalacc.append(loss)
  print('loss for f is ', loss)
print('final average is ', sum(finalacc)/len(finalacc))